In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

import pickle

In [2]:
# method used for saving object as pickle
def save_object_as_pickle(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file, pickle.HIGHEST_PROTOCOL)

## Import the data 

In [3]:
data_breast_cancer = datasets.load_breast_cancer(as_frame=True) 
print(data_breast_cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [4]:
# split the dataset
X = data_breast_cancer['data'][['mean texture', 'mean symmetry']]
y = data_breast_cancer['frame'].target
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# 3 bazowe + voting hard + voting soft - tj. razem 5 elementow 

In [6]:
X_train.head()

,mean texture,mean symmetry
68,17.33,0.2111
181,26.57,0.2395
63,13.86,0.2341
248,25.22,0.1897
60,14.88,0.2743


In [7]:
y_train.head()

68     1
181    0
63     1
248    1
60     1
Name: target, dtype: int32

# Ensemble

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

In [9]:
dtr_clf = DecisionTreeClassifier(random_state=42)
log_clf = LogisticRegression(random_state=42)
knn_clf = KNeighborsClassifier()

voting_clf_hard = VotingClassifier(
    estimators=[('lr', log_clf),
                ('dtr', dtr_clf),
                ('knn', knn_clf)],
    voting='hard')

voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf),
                ('dtr', dtr_clf),
                ('knn', knn_clf)],
    voting='soft')

### Calculate accuracy

In [10]:
acc_list = []

for clf in (dtr_clf, log_clf, knn_clf, voting_clf_hard, voting_clf_soft):
    clf.fit(X_train, y_train)
    y_pred_test = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    print("TRAIN", clf.__class__.__name__,
      train_acc)
    print("TEST", clf.__class__.__name__,
          test_acc)
    acc_list.append((train_acc, test_acc))
    
# WARNING! First voting classifier has 'hard' type of voting
# second voting classifier has 'soft' type of voting (it's not the same classifier!)

TRAIN DecisionTreeClassifier 1.0
TEST DecisionTreeClassifier 0.6140350877192983
TRAIN LogisticRegression 0.7230769230769231
TEST LogisticRegression 0.7017543859649122
TRAIN KNeighborsClassifier 0.7714285714285715
TEST KNeighborsClassifier 0.6403508771929824
TRAIN VotingClassifier 0.8351648351648352
TEST VotingClassifier 0.6929824561403509
TRAIN VotingClassifier 0.9648351648351648
TEST VotingClassifier 0.6666666666666666


In [11]:
# test if data is properly saved
for tup in acc_list:
    print(tup)

(1.0, 0.6140350877192983)
(0.7230769230769231, 0.7017543859649122)
(0.7714285714285715, 0.6403508771929824)
(0.8351648351648352, 0.6929824561403509)
(0.9648351648351648, 0.6666666666666666)


### Save data in the pickles

In [12]:
save_object_as_pickle(acc_list, "acc_vote.pkl")

clf_list = [dtr_clf, log_clf, knn_clf, voting_clf_hard, voting_clf_soft]

save_object_as_pickle(clf_list, "vote.pkl")

### Check if data saved properly

In [13]:
print("acc_vote.pkl\n", pd.read_pickle("acc_vote.pkl"), "\n")
print("vote.pkl\n", pd.read_pickle("vote.pkl"), "\n")

acc_vote.pkl
 [(1.0, 0.6140350877192983), (0.7230769230769231, 0.7017543859649122), (0.7714285714285715, 0.6403508771929824), (0.8351648351648352, 0.6929824561403509), (0.9648351648351648, 0.6666666666666666)] 

vote.pkl
 [DecisionTreeClassifier(random_state=42), LogisticRegression(random_state=42), KNeighborsClassifier(), VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('dtr', DecisionTreeClassifier(random_state=42)),
                             ('knn', KNeighborsClassifier())]), VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('dtr', DecisionTreeClassifier(random_state=42)),
                             ('knn', KNeighborsClassifier())],
                 voting='soft')] 



## 30 decision trees (ex. 5)
* Bagging,
* Bagging z wykorzystaniem 50% instancji,
* Pasting,
* Pasting z wykorzystaniem 50% instancji, oraz 
* Random Forest,
* AdaBoost,
* Gradient Boosting.


In [14]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [15]:
# Bagging
bgn_clf_full = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, bootstrap=True, random_state=42)
bgn_clf_half = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, bootstrap=True, max_samples=0.5, random_state=42)
# Pasting
pst_clf_full = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, bootstrap=False, random_state=42)
pst_clf_half = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, bootstrap=False, max_samples=0.5, random_state=42)
# Random forest
rnd_clf = RandomForestClassifier(n_estimators=30, random_state=42)
# Ada boost
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=30, random_state=42)
# Gradient boosting
gdb_clf = GradientBoostingClassifier(n_estimators=30, random_state=42)

In [16]:
classificators = [bgn_clf_full, bgn_clf_half, pst_clf_full, pst_clf_half, rnd_clf, ada_clf, gdb_clf]

clf_acc = []

for clf in classificators:
    clf.fit(X_train, y_train)
    y_pred_test = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    print("TRAIN", clf.__class__.__name__,
      train_acc)
    print("TEST", clf.__class__.__name__,
          test_acc, "\n")
    clf_acc.append((train_acc, test_acc))
    
# Bagging oraz Random forest daje takie same wyniki, dziwne - sprawdzic

TRAIN BaggingClassifier 0.9956043956043956
TEST BaggingClassifier 0.6754385964912281 

TRAIN BaggingClassifier 0.9296703296703297
TEST BaggingClassifier 0.6842105263157895 

TRAIN BaggingClassifier 1.0
TEST BaggingClassifier 0.6228070175438597 

TRAIN BaggingClassifier 0.9736263736263736
TEST BaggingClassifier 0.6491228070175439 

TRAIN RandomForestClassifier 0.9956043956043956
TEST RandomForestClassifier 0.6754385964912281 

TRAIN AdaBoostClassifier 1.0
TEST AdaBoostClassifier 0.631578947368421 

TRAIN GradientBoostingClassifier 0.8373626373626374
TEST GradientBoostingClassifier 0.7105263157894737 



### Save data in the pickles

In [17]:
save_object_as_pickle(clf_acc, "acc_bag.pkl")

save_object_as_pickle(classificators, "bag.pkl")

### Check if data saved properly

In [18]:
print("acc_bag.pkl\n", pd.read_pickle("acc_bag.pkl"), "\n")
print("bag.pkl\n", pd.read_pickle("bag.pkl"), "\n")

acc_bag.pkl
 [(0.9956043956043956, 0.6754385964912281), (0.9296703296703297, 0.6842105263157895), (1.0, 0.6228070175438597), (0.9736263736263736, 0.6491228070175439), (0.9956043956043956, 0.6754385964912281), (1.0, 0.631578947368421), (0.8373626373626374, 0.7105263157894737)] 

bag.pkl
 [BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=30,
                  random_state=42), BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=30, random_state=42), BaggingClassifier(base_estimator=DecisionTreeClassifier(), bootstrap=False,
                  n_estimators=30, random_state=42), BaggingClassifier(base_estimator=DecisionTreeClassifier(), bootstrap=False,
                  max_samples=0.5, n_estimators=30, random_state=42), RandomForestClassifier(n_estimators=30, random_state=42), AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=30,
                   random_state=42), GradientBoostingClassif

## Sampling

In [19]:
# Odnosnie zad 7: Tutaj BaggingClassifier z decision tree i max_features=2, 
# reszta ustawien domyslna

In [20]:
# prepare the data
X_all = data_breast_cancer['data']
y_all = data_breast_cancer['frame'].target
X_train, X_test, y_train, y_test= train_test_split(X_all, y_all, test_size=0.2, random_state=42)

In [21]:
X_train.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
68,9.029,17.33,58.79,250.5,0.10660,0.14130,0.31300,0.04375,0.2111,0.08046,...,10.31,22.65,65.50,324.7,0.14820,0.43650,1.25200,0.17500,0.4228,0.11750
181,21.090,26.57,142.70,1311.0,0.11410,0.28320,0.24870,0.14960,0.2395,0.07398,...,26.68,33.48,176.50,2089.0,0.14910,0.75840,0.67800,0.29030,0.4098,0.12840
63,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,0.06963,...,10.01,19.23,65.59,310.1,0.09836,0.16780,0.13970,0.05087,0.3282,0.08490
248,10.650,25.22,68.01,347.0,0.09657,0.07234,0.02379,0.01615,0.1897,0.06329,...,12.25,35.19,77.98,455.7,0.14990,0.13980,0.11250,0.06136,0.3409,0.08147
60,10.170,14.88,64.55,311.9,0.11340,0.08061,0.01084,0.01290,0.2743,0.06960,...,11.02,17.45,69.86,368.6,0.12750,0.09866,0.02168,0.02579,0.3557,0.08020


In [22]:
y_train.head()

68     1
181    0
63     1
248    1
60     1
Name: target, dtype: int32

In [23]:
smp_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, 
                            max_samples=0.5, max_features=2, 
                            bootstrap=True, bootstrap_features=False,
                            random_state=42)

In [24]:
smp_clf.fit(X_train, y_train)
y_pred_test = smp_clf.predict(X_test)
y_pred_train = smp_clf.predict(X_train)
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
print("TRAIN", smp_clf.__class__.__name__,
  train_acc)
print("TEST", smp_clf.__class__.__name__,
      test_acc, "\n")

smp_acc = [train_acc, test_acc]

TRAIN BaggingClassifier 0.9912087912087912
TEST BaggingClassifier 0.9736842105263158 



### Save data in the pickles

In [25]:
save_object_as_pickle(smp_acc, "acc_fea.pkl")

save_object_as_pickle(smp_clf, "fea.pkl")

### Check if data saved properly

In [26]:
print("acc_fea.pkl\n", pd.read_pickle("acc_fea.pkl"), "\n")
print("fea.pkl\n", pd.read_pickle("fea.pkl"), "\n")

acc_fea.pkl
 [0.9912087912087912, 0.9736842105263158] 

fea.pkl
 BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_features=2,
                  max_samples=0.5, n_estimators=30, random_state=42) 



## Classifier ranking

In [27]:
smp_clf.estimators_

[DecisionTreeClassifier(random_state=1952926171),
 DecisionTreeClassifier(random_state=1761383086),
 DecisionTreeClassifier(random_state=1449071958),
 DecisionTreeClassifier(random_state=1910541088),
 DecisionTreeClassifier(random_state=1341730541),
 DecisionTreeClassifier(random_state=1286572245),
 DecisionTreeClassifier(random_state=1005142668),
 DecisionTreeClassifier(random_state=502852014),
 DecisionTreeClassifier(random_state=186414760),
 DecisionTreeClassifier(random_state=1956263048),
 DecisionTreeClassifier(random_state=15592051),
 DecisionTreeClassifier(random_state=1628376228),
 DecisionTreeClassifier(random_state=1638437331),
 DecisionTreeClassifier(random_state=116435712),
 DecisionTreeClassifier(random_state=588556688),
 DecisionTreeClassifier(random_state=358068376),
 DecisionTreeClassifier(random_state=67998415),
 DecisionTreeClassifier(random_state=825108120),
 DecisionTreeClassifier(random_state=1237545031),
 DecisionTreeClassifier(random_state=1708477288),
 DecisionT

In [28]:
smp_clf.estimators_features_

[array([25, 17]),
 array([0, 6]),
 array([16, 19]),
 array([ 9, 16]),
 array([23,  9]),
 array([ 0, 12]),
 array([22,  4]),
 array([17, 13]),
 array([ 1, 25]),
 array([ 3, 20]),
 array([26, 21]),
 array([20, 13]),
 array([18, 24]),
 array([12, 20]),
 array([26, 29]),
 array([29,  3]),
 array([ 4, 12]),
 array([9, 1]),
 array([14, 21]),
 array([27, 17]),
 array([22, 18]),
 array([ 1, 15]),
 array([12, 14]),
 array([8, 4]),
 array([16,  5]),
 array([13,  9]),
 array([0, 5]),
 array([ 8, 17]),
 array([21, 19]),
 array([16, 13])]

In [29]:
# list of (train_accuracy, test_accuracy) elements
rank_acc = []
# list of [first_feautre_name, second_feauter_name] elements
rank_fea = []

for clf, fea_pair in zip(smp_clf.estimators_, smp_clf.estimators_features_):
#     clf.fit(X_train, y_train)
    first_fea = X_all.columns[fea_pair[0]]
    second_fea = X_all.columns[fea_pair[1]]
    columns = [first_fea, second_fea]
    y_pred_test = clf.predict(X_test[columns])
    y_pred_train = clf.predict(X_train[columns])
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    print("TRAIN", clf.__class__.__name__,
      train_acc)
    print("TEST", clf.__class__.__name__,
          test_acc, "")
    rank_acc.append((train_acc, test_acc))
    print("Features: ", first_fea, " ", second_fea, "\n")
    rank_fea.append([first_fea, second_fea])

TRAIN DecisionTreeClassifier 0.8307692307692308
TEST DecisionTreeClassifier 0.7456140350877193 
Features:  worst compactness   concave points error 

TRAIN DecisionTreeClassifier 0.9164835164835164
TEST DecisionTreeClassifier 0.8771929824561403 
Features:  mean radius   mean concavity 

TRAIN DecisionTreeClassifier 0.8
TEST DecisionTreeClassifier 0.6666666666666666 
Features:  concavity error   fractal dimension error 

TRAIN DecisionTreeClassifier 0.8043956043956044
TEST DecisionTreeClassifier 0.631578947368421 
Features:  mean fractal dimension   concavity error 

TRAIN DecisionTreeClassifier 0.9318681318681319
TEST DecisionTreeClassifier 0.9035087719298246 
Features:  worst area   mean fractal dimension 

TRAIN DecisionTreeClassifier 0.8725274725274725
TEST DecisionTreeClassifier 0.8947368421052632 
Features:  mean radius   perimeter error 

TRAIN DecisionTreeClassifier 0.9384615384615385
TEST DecisionTreeClassifier 0.9210526315789473 
Features:  worst perimeter   mean smoothness 



In [30]:
# prepare dataframe columns
train_acc= [x[0] for x in rank_acc] 
test_acc= [x[1] for x in rank_acc]
ranking = pd.DataFrame({
    "train accuracy": train_acc,
    "test accuracy": test_acc,
    "features": rank_fea
})

In [31]:
ranking = ranking.sort_values(by=['test accuracy', 'train accuracy'], ascending=False)

In [32]:
ranking

,train accuracy,test accuracy,features
9,0.929670,0.938596,"[mean area, worst radius]"
6,0.938462,0.921053,"[worst perimeter, mean smoothness]"
19,0.934066,0.921053,"[worst concave points, concave points error]"
13,0.925275,0.921053,"[perimeter error, worst radius]"
11,0.927473,0.912281,"[worst radius, area error]"
4,0.931868,0.903509,"[worst area, mean fractal dimension]"
20,0.927473,0.903509,"[worst perimeter, symmetry error]"
5,0.872527,0.894737,"[mean radius, perimeter error]"
26,0.918681,0.877193,"[mean radius, mean compactness]"
1,0.916484,0.877193,"[mean radius, mean concavity]"


### Save data in the pickles

In [33]:
save_object_as_pickle(ranking, "acc_fea_rank.pkl")

## Check if data saved properly

In [34]:
print("acc_fea_rank.pkl\n", pd.read_pickle("acc_fea_rank.pkl"), "\n")

acc_fea_rank.pkl
     train accuracy  test accuracy  \
9         0.929670       0.938596   
6         0.938462       0.921053   
19        0.934066       0.921053   
13        0.925275       0.921053   
11        0.927473       0.912281   
4         0.931868       0.903509   
20        0.927473       0.903509   
5         0.872527       0.894737   
26        0.918681       0.877193   
1         0.916484       0.877193   
15        0.918681       0.868421   
29        0.879121       0.859649   
7         0.832967       0.850877   
25        0.859341       0.842105   
14        0.898901       0.833333   
16        0.854945       0.807018   
10        0.863736       0.789474   
22        0.841758       0.789474   
0         0.830769       0.745614   
8         0.830769       0.728070   
27        0.791209       0.728070   
21        0.767033       0.684211   
2         0.800000       0.666667   
24        0.850549       0.640351   
17        0.771429       0.640351   
3         0.804396  

# Summary

In [35]:
print("acc_fea_rank.pkl\n", pd.read_pickle("acc_fea_rank.pkl"), "\n")
print("acc_vote.pkl\n", pd.read_pickle("acc_vote.pkl"), "\n")
print("vote.pkl\n", pd.read_pickle("vote.pkl"), "\n")
print("acc_bag.pkl\n", pd.read_pickle("acc_bag.pkl"), "\n")
print("bag.pkl\n", pd.read_pickle("bag.pkl"), "\n")
print("acc_fea.pkl\n", pd.read_pickle("acc_fea.pkl"), "\n")
print("fea.pkl\n", pd.read_pickle("fea.pkl"), "\n")
print("acc_fea_rank.pkl\n", pd.read_pickle("acc_fea_rank.pkl"), "\n")

acc_fea_rank.pkl
     train accuracy  test accuracy  \
9         0.929670       0.938596   
6         0.938462       0.921053   
19        0.934066       0.921053   
13        0.925275       0.921053   
11        0.927473       0.912281   
4         0.931868       0.903509   
20        0.927473       0.903509   
5         0.872527       0.894737   
26        0.918681       0.877193   
1         0.916484       0.877193   
15        0.918681       0.868421   
29        0.879121       0.859649   
7         0.832967       0.850877   
25        0.859341       0.842105   
14        0.898901       0.833333   
16        0.854945       0.807018   
10        0.863736       0.789474   
22        0.841758       0.789474   
0         0.830769       0.745614   
8         0.830769       0.728070   
27        0.791209       0.728070   
21        0.767033       0.684211   
2         0.800000       0.666667   
24        0.850549       0.640351   
17        0.771429       0.640351   
3         0.804396  